<a href="https://colab.research.google.com/github/plodha/AlternusVera/blob/master/version2/preprocessing/Merge_combinedDatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task:

* This colab is for merging multiple data sources and creating combined csv file
* Remove everything else

# Description


This colab is being used to prepare Calfornia baseline analysis data for all counties. We are prepapreing here two sets of data. 

* 1. CA_combined data from merging all the listed datasets- 
> * Merge Social Distancing, 
> * mobility data,
> * Covid Cases, 
> * Population Density 
> * Mask data for rule

* 2. Merging Social Distancing Inertia (Maryland data) and Covid data integration

In [ ]:
!pip install wget


  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=207953f70d65b826a90e4f316f2a70ca216e75559b1feb15a6c84a953acf3a75
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls  '/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/'

CA
CA_combined_Feb04.csv
CA_combined_Sep09.csv
CA_combined_Sep25.csv
CA_Covid-19.csv
CA-CovidAug22.csv
CA_labeledData_Oct09.csv
CA_labeledData_Oct10.csv
CA_labeledData_Sep09
CA_labeledData_Sep18.csv
CA_labeledData_Sep20.csv
CA_labeledData_Sep22.csv
CA_labeledData_Sep25.csv
CA_labeledData_Sep26.csv
CA_labeledData_Sep29.csv
CA_sd_inertia_labeledData_Sep09
CA_sd_inertia_labeledData_Sep18.csv
CA_sd_inertia_labeledData_Sep20.csv
CA_sd_inertia_labeledData_Sep22.csv
CA_sd_inertia_labeledData_Sep25.csv
CA_socialDistancingInertiaData_Sep09.csv
Maryland_Inertia_Data
Maryland_Inertia_DataMarylan_inertia_clusterlabeled_Oct20.csv
mask_rule.csv
mask_rule_data.csv
Newyork_combinedAug22.csv
Newyork_combined.csv
Newyork_combinedSep24.csv
NY_Covid-19.csv
NY-CovidAug21.csv
NY-CovidAug22.csv
NY_labeledData_Aug23.csv
NY_labeledData_Aug27.csv
NY_labeledData_Sep13.csv
NY_labeledData_Sep26.csv
NY_OneCountyLabeledData_forR_Oct18.csv
NY_sd_inertia_labeledData_Aug26.csv
NY_sd_inertia_labeledData_Aug27.csv
NY_sd_

# Load library

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import wget
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
from sklearn.mixture import GaussianMixture
from datetime import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import warnings; warnings.simplefilter('ignore')

# 1 Load Data Sets

## 1.1 County Population

In [ ]:
US_population_filename = wget.download('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv')
county_population_US = pd.read_csv(US_population_filename,low_memory=False)
print(county_population_US.shape)

county_population_US.head(2)

(3195, 4)


,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869


In [ ]:
CA_population = county_population_US[county_population_US['State'] == 'CA']
print(CA_population.shape)
CA_population.head(2)

(60, 4)


,countyFIPS,County Name,State,population
191,0,Statewide Unallocated,CA,0
192,6000,Grand Princess Cruise Ship,CA,0


In [ ]:
CA_population['County Name'].unique()

array(['Statewide Unallocated', 'Grand Princess Cruise Ship',
       'Alameda County', 'Alpine County', 'Amador County', 'Butte County',
       'Calaveras County', 'Colusa County', 'Contra Costa County',
       'Del Norte County', 'El Dorado County', 'Fresno County',
       'Glenn County', 'Humboldt County', 'Imperial County',
       'Inyo County', 'Kern County', 'Kings County', 'Lake County',
       'Lassen County', 'Los Angeles County', 'Madera County',
       'Marin County', 'Mariposa County', 'Mendocino County',
       'Merced County', 'Modoc County', 'Mono County', 'Monterey County',
       'Napa County', 'Nevada County', 'Orange County', 'Placer County',
       'Plumas County', 'Riverside County', 'Sacramento County',
       'San Benito County', 'San Bernardino County', 'San Diego County',
       'San Francisco County', 'San Joaquin County',
       'San Luis Obispo County', 'San Mateo County',
       'Santa Barbara County', 'Santa Clara County', 'Santa Cruz County',
       'Shast

In [ ]:
LA_county_population = CA_population[CA_population['County Name'] == 'Los Angeles County']
LA_county_population

,countyFIPS,County Name,State,population
211,6037,Los Angeles County,CA,10039107


## 1.2 Land Area 

In [ ]:
landarea_filename = wget.download('https://raw.githubusercontent.com/ykzeng/covid-19/master/data/census-landarea-all.csv')
landarea_data = pd.read_csv(landarea_filename,low_memory=False)
print(landarea_data.shape)


(3195, 52)


In [ ]:
landarea_data.head(2)

,fips,PST045212,PST040210,PST120212,POP010210,AGE135212,AGE295212,AGE775212,SEX255212,RHI125212,RHI225212,RHI325212,RHI425212,RHI525212,RHI625212,RHI725212,RHI825212,POP715211,POP645211,POP815211,EDU635211,EDU685211,VET605211,LFE305211,HSG010211,HSG445211,HSG096211,HSG495211,HSD410211,HSD310211,INC910211,INC110211,PVY020211,BZA010211,BZA110211,BZA115211,NES010211,SBO001207,SBO315207,SBO115207,SBO215207,SBO515207,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030212,LND110210,POP060210
0,0,313914040,308747508,1.7,308745538,6.4,23.5,13.7,50.8,77.9,13.1,1.2,5.1,0.2,2.4,16.9,63.0,84.6,12.8,20.3,85.4,28.2,22215303,25.4,132312404,66.1,25.9,186200,114761359,2.60,27915,52762,14.3,7354043,113425965,1.3,22491080,27092908,7.1,0.9,5.7,0.1,8.3,28.8,5319456312,4174286516,3917663456,12990,613795732,829658,3531905.43,87.4
1,1000,4822023,4779745,0.9,4779736,6.3,23.3,14.5,51.5,70.0,26.5,0.7,1.2,0.1,1.5,4.1,66.6,84.5,3.4,5.0,81.9,22.0,403982,24.0,2182088,70.7,15.5,120800,1831269,2.53,23483,42934,17.6,97743,1573138,0.3,321641,382350,14.8,0.8,1.8,0.1,1.2,28.1,112858843,52252752,57344851,12364,6426342,13506,50645.33,94.4


In [ ]:
landarea_data['fips'].unique()

array([    0,  1000,  1001, ..., 56041, 56043, 56045])

In [ ]:
#LND110210

landarea_data[landarea_data['fips'] == 36081]['LND110210']

1902    108.53
Name: LND110210, dtype: float64

#### Population density

In [ ]:
density = 2253858/108.53

density

20767.142725513684

LA population density

In [ ]:
LA_population_density=pd.merge(LA_county_population,landarea_data,how='inner', left_on=['countyFIPS'], right_on = ['fips'])

In [ ]:
LA_population_density['population_density']= LA_population_density['population']/LA_population_density['LND110210']
LA_population_density = LA_population_density[['fips','County Name', 'State', 'population','LND110210','population_density']].copy()
LA_population_density.tail(3)

,fips,County Name,State,population,LND110210,population_density
0,6037,Los Angeles County,CA,10039107,4057.88,2473.978284


## 1.3 Mask data

In [ ]:
mask_rule_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/mask_rule.csv',low_memory=False)
 
mask_rule_data.head()

,state_name,Law Date,Masks Required?,Type of Requirement,mask_required_rule,requirement_code
0,Louisiana,2020-07-13,Entire State,Everywhere in public where social distancing i...,0,3
1,Kentucky,2020-07-10,Entire State,Everywhere in public where social distancing i...,0,3
2,West Virginia,2020-07-07,Entire State,All Public Indoor Spaces (where social distanc...,0,1
3,Texas,2020-07-03,Entire State,Everywhere in public where social distancing i...,0,4
4,Kansas,2020-07-03,Entire State,Everywhere in public where social distancing i...,0,3


In [ ]:
CA_mask_rule = mask_rule_data[mask_rule_data['state_name'] == 'California']
CA_mask_rule

,state_name,Law Date,Masks Required?,Type of Requirement,mask_required_rule,requirement_code
9,California,2020-06-18,Entire State,Everywhere in public where social distancing i...,0,3


In [ ]:
mask_filename = wget.download('https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv')
mask_filename

'mask-use-by-county.csv'

In [ ]:

mask_data = pd.read_csv(mask_filename,low_memory=False)
print(mask_data.shape)

(3142, 6)


In [ ]:
mask_data.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [ ]:
#los angeles
mask_data[mask_data['COUNTYFP'] == 6037]

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
204,6037,0.021,0.013,0.049,0.131,0.786


## 1.4 Mobility Data 

### Google mobility data

In [ ]:
google_data_url = 'https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/google_reports/mobility_report_US.csv'

google_filename = wget.download(google_data_url)
google_mobility_data = pd.read_csv(google_filename,low_memory=False)
google_mobility_data.head(2)

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential
0,Total,Total,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,Total,Total,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0


In [ ]:
CA_GoogleMobility_data = google_mobility_data[google_mobility_data['state'] == 'California']
print(CA_GoogleMobility_data.shape)
CA_GoogleMobility_data.tail(2)

(19533, 9)


,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential
72355,California,Yuba County,2021-01-30,-13.0,-7.0,22.0,-27.0,-9.0,5.0
72356,California,Yuba County,2021-01-31,-10.0,-7.0,45.0,NaN,-12.0,4.0


### Apple mobility data

In [ ]:
apple_report_url = 'https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/apple_reports/apple_mobility_report_US.csv'

apple_filename = wget.download(apple_report_url)
apple_mobility_data = pd.read_csv(apple_filename,low_memory=False)
apple_mobility_data.tail(2)

,state,county_and_city,geo_type,date,driving,transit,walking
869452,Wyoming,Uinta County,county,2021-02-01,19.08,NaN,NaN
869453,Wyoming,Uinta County,county,2021-02-02,15.98,NaN,NaN


In [ ]:
CA_AppleMobility_data = apple_mobility_data[apple_mobility_data['state'] == 'California']
print(CA_AppleMobility_data.shape)
CA_AppleMobility_data.tail(2)

(28939, 7)


,state,county_and_city,geo_type,date,driving,transit,walking
78648,California,Yuba County,county,2021-02-01,7.06,NaN,NaN
78649,California,Yuba County,county,2021-02-02,11.58,NaN,NaN


In [ ]:
CA_AppleMobility_data['county'] = CA_AppleMobility_data['county_and_city']

In [ ]:
CA_AppleMobility_data = CA_AppleMobility_data[['state','county','date','driving','transit','walking']]

CA_AppleMobility_data.tail(2)

,state,county,date,driving,transit,walking
78648,California,Yuba County,2021-02-01,7.06,NaN,NaN
78649,California,Yuba County,2021-02-02,11.58,NaN,NaN


In [ ]:
CA_mobility_data = pd.merge(CA_GoogleMobility_data,CA_AppleMobility_data,how='outer' ,on=['state','county','date'])
CA_mobility_data.shape

(29743, 12)

In [ ]:
CA_mobility_data = CA_mobility_data.fillna(0)
CA_mobility_data.tail()

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
29738,California,Yuba County,2020-02-12,0.0,0.0,0.0,0.0,0.0,0.0,10.18,0.0,0.0
29739,California,Yuba County,2020-02-13,0.0,0.0,0.0,0.0,0.0,0.0,17.75,0.0,0.0
29740,California,Yuba County,2020-02-14,0.0,0.0,0.0,0.0,0.0,0.0,40.51,0.0,0.0
29741,California,Yuba County,2021-02-01,0.0,0.0,0.0,0.0,0.0,0.0,7.06,0.0,0.0
29742,California,Yuba County,2021-02-02,0.0,0.0,0.0,0.0,0.0,0.0,11.58,0.0,0.0


### Los Angeles county mobility data

In [ ]:
LA_mobility_data = CA_mobility_data[CA_mobility_data['county'] == 'Los Angeles']
print(LA_mobility_data.shape)
LA_mobility_data.tail()

(385, 12)


,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
23363,California,Los Angeles,2021-01-29,0.0,0.0,0.0,0.0,0.0,0.0,-8.86,-62.08,-9.59
23364,California,Los Angeles,2021-01-30,0.0,0.0,0.0,0.0,0.0,0.0,-0.01,-58.36,13.52
23365,California,Los Angeles,2021-01-31,0.0,0.0,0.0,0.0,0.0,0.0,-19.84,-62.73,-12.67
23366,California,Los Angeles,2021-02-01,0.0,0.0,0.0,0.0,0.0,0.0,-14.93,-54.96,-11.99
23367,California,Los Angeles,2021-02-02,0.0,0.0,0.0,0.0,0.0,0.0,-11.60,-53.28,-7.27


In [ ]:
LA_mobility_data.head()

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
22983,California,Los Angeles,2020-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
22984,California,Los Angeles,2020-01-14,0.0,0.0,0.0,0.0,0.0,0.0,5.93,3.78,5.72
22985,California,Los Angeles,2020-01-15,0.0,0.0,0.0,0.0,0.0,0.0,12.23,5.28,11.35
22986,California,Los Angeles,2020-01-16,0.0,0.0,0.0,0.0,0.0,0.0,11.73,3.12,11.68
22987,California,Los Angeles,2020-01-17,0.0,0.0,0.0,0.0,0.0,0.0,30.20,9.34,34.45


## 1.5 Social Distancing metric 

In [ ]:
socialdistancing_url = 'https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-mobility-daterow.csv'

socialdistancing_file = wget.download(socialdistancing_url)
socialdistancing_data = pd.read_csv(socialdistancing_file,low_memory=False)
socialdistancing_data.tail(2)



,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
867800,2021-01-31,US,2,Wyoming,Uinta County,56041.0,679,2.386,76
867801,2021-02-01,US,2,Wyoming,Uinta County,56041.0,665,3.550,113


In [ ]:
newyork_socialdistancing_data= socialdistancing_data[(socialdistancing_data['admin1']=='California') &
                                           (socialdistancing_data['admin_level']== 2)]

newyork_socialdistancing_data.head()

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
52971,2020-03-01,US,2,California,Alameda County,6001.0,16737,3.684,58
52972,2020-03-02,US,2,California,Alameda County,6001.0,17269,6.182,98
52973,2020-03-03,US,2,California,Alameda County,6001.0,18181,6.271,100
52974,2020-03-04,US,2,California,Alameda County,6001.0,18495,6.209,99
52975,2020-03-05,US,2,California,Alameda County,6001.0,17496,6.654,106


In [ ]:
LA_social_distancing = newyork_socialdistancing_data[newyork_socialdistancing_data['admin2'] == 'Los Angeles County']
print(LA_social_distancing.shape)
LA_social_distancing.head()

(320, 9)


,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
58411,2020-03-01,US,2,California,Los Angeles County,6037.0,110713,3.996,60
58412,2020-03-02,US,2,California,Los Angeles County,6037.0,112327,6.502,98
58413,2020-03-03,US,2,California,Los Angeles County,6037.0,116225,6.591,99
58414,2020-03-04,US,2,California,Los Angeles County,6037.0,118658,6.595,100
58415,2020-03-05,US,2,California,Los Angeles County,6037.0,113023,7.008,106


# 2 Load Social Distancing Inertia Data [Maryland University]

In [ ]:
maryland_sd_inertia = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/Social_Distancing_Inertia_County.csv',low_memory=False)
maryland_sd_inertia.head()

,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases
0,1001,Autauga County,1,54,27,2.25,38.8,1.2,24.7,0.25,2.00,0,55601,-22.5,1/1/2020,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.3,0.0,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0
1,1001,Autauga County,1,25,18,2.83,43.5,0.9,37.6,0.62,2.21,0,55601,-14.3,1/2/2020,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.3,0.0,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0
2,1001,Autauga County,1,15,15,3.18,42.2,1.0,40.3,0.62,2.57,0,55601,-0.4,1/3/2020,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.3,0.0,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0
3,1001,Autauga County,1,33,20,2.85,43.9,1.2,29.5,0.33,2.52,0,55601,-2.3,1/4/2020,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.3,0.0,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0
4,1001,Autauga County,1,40,22,2.56,42.8,1.2,30.9,0.23,2.33,0,55601,-9.7,1/5/2020,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.3,0.0,0.0,0.0,0,47.1,0.0,0.0,0.0,0,0


## Fixdate column of Social Distancing inertia

In [ ]:
def fixDate(x, appendTwenty):
  arr = x.split('/')
  m = arr[0]
  d = arr[1]
  y = arr[2]

  if int(m) < 10:
    m = '0'+str(m)
  if int(d) < 10:
    d = '0'+str(d)
  if appendTwenty == True:
    return '20'+y+'-'+m+'-'+d
  else:
      return y+'-'+m+'-'+d

In [ ]:
maryland_sd_inertia['date'] = maryland_sd_inertia['date'].apply(lambda x: fixDate(x,False))
maryland_sd_inertia.reset_index(drop=True)

,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases
0,1001,Autauga County,1,54,27,2.25,38.8,1.2,24.7,0.25,2.00,0,55601,-22.5,2020-01-01,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.30,0.0,0.0000,0.00,0,47.100000,0.0,0.00,0.00,0,0
1,1001,Autauga County,1,25,18,2.83,43.5,0.9,37.6,0.62,2.21,0,55601,-14.3,2020-01-02,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.30,0.0,0.0000,0.00,0,47.100000,0.0,0.00,0.00,0,0
2,1001,Autauga County,1,15,15,3.18,42.2,1.0,40.3,0.62,2.57,0,55601,-0.4,2020-01-03,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.30,0.0,0.0000,0.00,0,47.100000,0.0,0.00,0.00,0,0
3,1001,Autauga County,1,33,20,2.85,43.9,1.2,29.5,0.33,2.52,0,55601,-2.3,2020-01-04,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.30,0.0,0.0000,0.00,0,47.100000,0.0,0.00,0.00,0,0
4,1001,Autauga County,1,40,22,2.56,42.8,1.2,30.9,0.23,2.33,0,55601,-9.7,2020-01-05,0.11,19,58786,19.0,2.8,48.68,91,19,101,3.79,0.33,0.025,39.0,12,0.0,0,0.9,2.7,3.2,0.30,0.0,0.0000,0.00,0,47.100000,0.0,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732081,56037,Sweetwater County,56,19,22,4.62,25.1,10.5,63.0,0.58,4.04,0,43051,37.8,2020-08-20,2.52,17,73008,0.8,16.0,51.87,4,3,134,3.76,0.50,0.017,47.0,6,17.1,105,3.8,12.6,26.2,1.51,0.3,0.0542,2.19,0,37.716682,5.4,111.08,1.31,3,90
732082,56039,Teton County,56,33,21,4.12,33.0,20.7,30.2,0.39,3.73,4,23081,0.8,2020-08-20,4.05,20,83831,1.1,14.9,51.65,5,7,297,3.76,0.50,0.017,47.0,16,17.1,105,3.8,12.6,26.2,1.51,0.3,0.1155,2.19,0,37.716682,5.4,111.08,1.31,3,100
732083,56041,Uinta County,56,21,20,4.44,36.8,24.5,79.0,0.65,3.79,0,20299,52.1,2020-08-20,3.60,18,58235,0.1,9.1,50.97,10,5,147,3.76,0.50,0.017,47.0,9,17.1,105,3.8,12.6,26.2,1.51,0.3,0.0985,2.19,14,37.716682,5.4,111.08,1.31,3,80
732084,56043,Washakie County,56,35,41,3.34,19.0,2.1,59.4,0.85,2.50,2,7885,20.9,2020-08-20,0.03,28,53426,0.0,14.2,50.89,4,3,205,3.76,0.50,0.017,47.0,73,17.1,105,3.8,12.6,26.2,1.51,0.3,0.2536,2.19,5,37.716682,5.4,111.08,1.31,3,7


# 3.Combining CA -all county data 

In [ ]:
CA_covidcases = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/CA-CovidAug22.csv', low_memory =False)
CA_covidcases.tail()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
12562,2020-08-17,California,90006.0,0,0,0,0
12563,2020-08-18,California,90006.0,0,0,0,0
12564,2020-08-19,California,90006.0,0,0,0,0
12565,2020-08-20,California,90006.0,0,0,0,0
12566,2020-08-21,California,90006.0,0,0,0,0


In [ ]:
#CA_covidcases['Date'] = CA_covidcases['Date'].apply(lambda x: fixDate(x))
CA_covidcases.reset_index(drop=True)
CA_covidcases.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
0,2020-01-22,California,6001.0,0,0,0,0
1,2020-01-23,California,6001.0,0,0,0,0


In [ ]:
CA_covidcases['FIPS'] = CA_covidcases['FIPS'].apply(lambda x :int(x))

In [ ]:
CA_covidcases.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
0,2020-01-22,California,6001,0,0,0,0
1,2020-01-23,California,6001,0,0,0,0


In [ ]:
CA_covidcases.shape

(12567, 7)

## Combining mask data -CA county

In [ ]:
CA_covidcases['mask_rule_active'] = CA_covidcases['Date'].apply(lambda x : 1 if x >= CA_mask_rule['Law Date'].values[0] else 0 )


In [ ]:
#Mask
CA_mask = pd.merge(CA_covidcases,mask_data,how='inner',left_on=['FIPS'],right_on=['COUNTYFP'])
CA_mask.head(2)


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,2020-01-22,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795
1,2020-01-23,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795


In [ ]:
CA_mask.shape

(12354, 14)

In [ ]:
def updateMaskWearingPercent(x):
  #print(x)
  if x.mask_rule_active == 1:
    return x.ALWAYS
  else:
    return 0.0

In [ ]:
CA_mask['mask_wearing_percent'] = CA_mask.apply(lambda x : updateMaskWearingPercent(x),axis=1)
 
CA_mask.tail()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,mask_wearing_percent
12349,2020-08-17,California,6115,758,4,0,0,1,6115,0.057,0.035,0.071,0.167,0.669,0.669
12350,2020-08-18,California,6115,786,4,28,0,1,6115,0.057,0.035,0.071,0.167,0.669,0.669
12351,2020-08-19,California,6115,821,4,35,0,1,6115,0.057,0.035,0.071,0.167,0.669,0.669
12352,2020-08-20,California,6115,821,4,0,0,1,6115,0.057,0.035,0.071,0.167,0.669,0.669
12353,2020-08-21,California,6115,829,4,8,0,1,6115,0.057,0.035,0.071,0.167,0.669,0.669


In [ ]:
CA_mask.head()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,mask_wearing_percent
0,2020-01-22,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0
1,2020-01-23,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0
2,2020-01-24,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0
3,2020-01-25,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0
4,2020-01-26,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0


## Combining population density data

In [ ]:
# Pop density
#landarea_data[landarea_data['fips'] == 36081]['LND110210']

CA_landarea_df = pd.merge(CA_mask,landarea_data,how='inner',left_on=['FIPS'],right_on=['fips'])
CA_landarea_df.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,mask_wearing_percent,fips,PST045212,PST040210,PST120212,POP010210,AGE135212,AGE295212,AGE775212,SEX255212,RHI125212,RHI225212,RHI325212,RHI425212,RHI525212,RHI625212,RHI725212,RHI825212,POP715211,POP645211,POP815211,EDU635211,EDU685211,VET605211,LFE305211,HSG010211,HSG445211,HSG096211,HSG495211,HSD410211,HSD310211,INC910211,INC110211,PVY020211,BZA010211,BZA110211,BZA115211,NES010211,SBO001207,SBO315207,SBO115207,SBO215207,SBO515207,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030212,LND110210,POP060210
0,2020-01-22,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0,6001,1554720,1510271,2.9,1510271,6.3,22.1,11.8,51.0,52.3,12.8,1.2,27.6,1.0,5.1,22.7,33.5,84.0,30.7,42.8,86.0,40.8,68414,28.1,584622,54.5,37.9,558300,536160,2.73,34937,70821,11.8,35921,567151,0.8,118341,140342,6.8,0.0,24.4,0.0,8.9,32.2,25236865,35674007,17909054,12359,2807766,2745,739.02,2043.6
1,2020-01-23,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0,6001,1554720,1510271,2.9,1510271,6.3,22.1,11.8,51.0,52.3,12.8,1.2,27.6,1.0,5.1,22.7,33.5,84.0,30.7,42.8,86.0,40.8,68414,28.1,584622,54.5,37.9,558300,536160,2.73,34937,70821,11.8,35921,567151,0.8,118341,140342,6.8,0.0,24.4,0.0,8.9,32.2,25236865,35674007,17909054,12359,2807766,2745,739.02,2043.6


In [ ]:
CA_landarea_df.shape

(12354, 67)

In [ ]:
CA_landarea = pd.merge(CA_landarea_df,CA_population,how='inner',left_on=['FIPS'],right_on=['countyFIPS'])

CA_landarea.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,mask_wearing_percent,fips,PST045212,PST040210,PST120212,POP010210,AGE135212,AGE295212,AGE775212,SEX255212,RHI125212,RHI225212,RHI325212,RHI425212,RHI525212,RHI625212,RHI725212,RHI825212,POP715211,POP645211,POP815211,EDU635211,EDU685211,VET605211,LFE305211,HSG010211,HSG445211,HSG096211,HSG495211,HSD410211,HSD310211,INC910211,INC110211,PVY020211,BZA010211,BZA110211,BZA115211,NES010211,SBO001207,SBO315207,SBO115207,SBO215207,SBO515207,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030212,LND110210,POP060210,countyFIPS,County Name,State,population
0,2020-01-22,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0,6001,1554720,1510271,2.9,1510271,6.3,22.1,11.8,51.0,52.3,12.8,1.2,27.6,1.0,5.1,22.7,33.5,84.0,30.7,42.8,86.0,40.8,68414,28.1,584622,54.5,37.9,558300,536160,2.73,34937,70821,11.8,35921,567151,0.8,118341,140342,6.8,0.0,24.4,0.0,8.9,32.2,25236865,35674007,17909054,12359,2807766,2745,739.02,2043.6,6001,Alameda County,CA,1671329
1,2020-01-23,California,6001,0,0,0,0,0,6001,0.019,0.008,0.055,0.123,0.795,0.0,6001,1554720,1510271,2.9,1510271,6.3,22.1,11.8,51.0,52.3,12.8,1.2,27.6,1.0,5.1,22.7,33.5,84.0,30.7,42.8,86.0,40.8,68414,28.1,584622,54.5,37.9,558300,536160,2.73,34937,70821,11.8,35921,567151,0.8,118341,140342,6.8,0.0,24.4,0.0,8.9,32.2,25236865,35674007,17909054,12359,2807766,2745,739.02,2043.6,6001,Alameda County,CA,1671329


In [ ]:
CA_landarea.shape

(12354, 71)

In [ ]:
CA_landarea['population_density']= CA_landarea['population']/CA_landarea['LND110210']
CA_landarea = CA_landarea[['Date','fips','New cases','County Name','State','mask_rule_active','mask_wearing_percent',  'population','LND110210','population_density']].copy()
CA_landarea.tail(3)

,Date,fips,New cases,County Name,State,mask_rule_active,mask_wearing_percent,population,LND110210,population_density
12351,2020-08-19,6115,35,Yuba County,CA,1,0.669,78668,631.84,124.506204
12352,2020-08-20,6115,0,Yuba County,CA,1,0.669,78668,631.84,124.506204
12353,2020-08-21,6115,8,Yuba County,CA,1,0.669,78668,631.84,124.506204


## Combining mobility data 

In [ ]:
# Mobility 
CA_mobility_data.head(2)

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
0,California,Total,2020-02-15,1.0,1.0,19.0,1.0,-1.0,0.0,30.03,1.90,58.40
1,California,Total,2020-02-16,5.0,0.0,31.0,1.0,-1.0,-1.0,6.16,-11.45,25.46


In [ ]:
CA_mobility_combined = pd.merge(CA_landarea,CA_mobility_data,how='inner',left_on=['Date','County Name'],right_on=['date','county'])
CA_mobility_combined.head(2)

,Date,fips,New cases,County Name,State,mask_rule_active,mask_wearing_percent,population,LND110210,population_density,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
0,2020-01-22,6001,0,Alameda County,CA,0,0.0,1671329,739.02,2261.547725,California,Alameda County,2020-01-22,0.0,0.0,0.0,0.0,0.0,0.0,10.27,25.12,50.04
1,2020-01-23,6001,0,Alameda County,CA,0,0.0,1671329,739.02,2261.547725,California,Alameda County,2020-01-23,0.0,0.0,0.0,0.0,0.0,0.0,8.72,20.07,36.32


In [ ]:
CA_mobility_combined.shape

(11727, 22)

## Combining social distancing 

In [ ]:
newyork_socialdistancing_data.head(2)

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
52971,2020-03-01,US,2,California,Alameda County,6001.0,16737,3.684,58
52972,2020-03-02,US,2,California,Alameda County,6001.0,17269,6.182,98


In [ ]:
newyork_socialdistancing_data['fips'] = newyork_socialdistancing_data['fips'].apply(lambda x:int(x))
newyork_socialdistancing_data.tail(2)

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
70172,2021-01-31,US,2,California,Yuba County,6115,1023,0.635,6
70173,2021-02-01,US,2,California,Yuba County,6115,989,2.357,24


In [ ]:
CA_mobility_combined.shape

(11727, 22)

In [ ]:
CA_socialDistancing_combined = pd.merge(CA_mobility_combined,newyork_socialdistancing_data,how='inner',left_on=['Date','County Name'],right_on=['date','admin2'])
CA_socialDistancing_combined.head(2)

,Date,fips_x,New cases,County Name,State,mask_rule_active,mask_wearing_percent,population,LND110210,population_density,state,county,date_x,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking,date_y,country_code,admin_level,admin1,admin2,fips_y,samples,m50,m50_index
0,2020-03-01,6001,0,Alameda County,CA,0,0.0,1671329,739.02,2261.547725,California,Alameda County,2020-03-01,6.0,9.0,15.0,1.0,4.0,0.0,-7.65,-2.79,11.07,2020-03-01,US,2,California,Alameda County,6001,16737,3.684,58
1,2020-03-02,6001,0,Alameda County,CA,0,0.0,1671329,739.02,2261.547725,California,Alameda County,2020-03-02,4.0,7.0,7.0,-1.0,3.0,0.0,6.76,6.53,13.48,2020-03-02,US,2,California,Alameda County,6001,17269,6.182,98


In [ ]:
CA_socialDistancing_combined.shape

(9191, 31)

## 3.5 Combine Population Density data from 2020

In [ ]:
PopDensity = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/population-density/Average_Household_Size_and_Population_Density_County.csv', low_memory=False)

In [ ]:
pop_density_2020_california = PopDensity[PopDensity['State'] == 'California']
pop_density_2020_california.head()

,FID,COUNTYNS,GEOID,ALAND,AWATER,NAME,State,B25010_001E,B25010_001M,B25010_002E,B25010_002M,B25010_003E,B25010_003M,B01001_001E,B01001_001M,B01001_calc_PopDensity,created_user,created_date,last_edited_user,last_edited_date,B01001_calc_PopDensityM,SHAPE_Length,SHAPE_Area
186,187,1675839,6001,1909598013,216923745,Alameda County,California,2.81,0.01,2.93,0.02,2.68,0.02,1643700,NaN,860.757075,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,2.677466,0.199990
187,188,1675840,6003,1912292630,12557304,Alpine County,California,3.71,1.02,3.66,0.97,4.00,2.65,1146,161.0,0.599281,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.084192,2.174214,0.199044
188,189,1675841,6005,1539933577,29470567,Amador County,California,2.42,0.07,2.43,0.08,2.40,0.19,37829,NaN,24.565345,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,2.909839,0.161954
189,190,1675842,6007,4238438186,105311003,Butte County,California,2.55,0.03,2.48,0.04,2.64,0.05,227075,NaN,53.575159,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,4.205815,0.455942
190,191,1675885,6009,2641829200,43797659,Calaveras County,California,2.57,0.07,2.49,0.08,2.83,0.22,45235,NaN,17.122606,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,2.961851,0.276235


In [ ]:
CA_socialDistancing_combined_pop = CA_socialDistancing_combined

In [ ]:
CA_socialDistancing_combined_pop['PopDensity'] = 0.00
for index, row in CA_socialDistancing_combined_pop.iterrows():
  county_fips = CA_socialDistancing_combined_pop['fips_x'][index]
  temp = pop_density_2020_california[pop_density_2020_california.eq(county_fips).any(1)].reset_index()
  CA_socialDistancing_combined_pop['PopDensity'][index] = temp['B01001_calc_PopDensity'][0]

In [ ]:
print(CA_socialDistancing_combined_pop.shape)

CA_socialDistancing_combined_pop.head(2)

(9191, 32)


,Date,fips_x,New cases,County Name,State,mask_rule_active,mask_wearing_percent,population,LND110210,population_density,state,county,date_x,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking,date_y,country_code,admin_level,admin1,admin2,fips_y,samples,m50,m50_index,PopDensity
0,2020-03-01,6001,0,Alameda County,CA,0,0.0,1671329,739.02,2261.547725,California,Alameda County,2020-03-01,6.0,9.0,15.0,1.0,4.0,0.0,-7.65,-2.79,11.07,2020-03-01,US,2,California,Alameda County,6001,16737,3.684,58,860.757075
1,2020-03-02,6001,0,Alameda County,CA,0,0.0,1671329,739.02,2261.547725,California,Alameda County,2020-03-02,4.0,7.0,7.0,-1.0,3.0,0.0,6.76,6.53,13.48,2020-03-02,US,2,California,Alameda County,6001,17269,6.182,98,860.757075


## Filtering columns for feature selection

In [70]:
columns = ['Date','fips_x','County Name','population','retail and recreation','grocery and pharmacy','parks','transit stations','workplaces','residential','driving','m50','m50_index','mask_rule_active','mask_wearing_percent','New cases', 'PopDensity']
CA_All_combined = CA_socialDistancing_combined_pop[columns]

CA_All_combined.head(2)


,Date,fips_x,County Name,population,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,m50,m50_index,mask_rule_active,mask_wearing_percent,New cases,PopDensity
0,2020-03-01,6001,Alameda County,1671329,6.0,9.0,15.0,1.0,4.0,0.0,-7.65,3.684,58,0,0.0,0,860.757075
1,2020-03-02,6001,Alameda County,1671329,4.0,7.0,7.0,-1.0,3.0,0.0,6.76,6.182,98,0,0.0,0,860.757075


In [71]:
CA_All_combined.shape

(9191, 17)

### Combined file save-CA

In [72]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/'
filename = 'CA_combined_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
CA_All_combined.to_csv(location+filename,index=False)


CA_combined_Feb04.csv


# 4.Merging Social Distancing Inertia (Maryland data) and Covid data integration

In [ ]:
CA_covidcases.dtypes

Date                object
Province_State      object
FIPS                 int64
Confirmed            int64
Deaths               int64
New cases            int64
New deaths           int64
mask_rule_active     int64
dtype: object

In [ ]:
print(maryland_sd_inertia.shape)
CA_covidcases.shape

(732086, 45)


(12567, 8)

In [ ]:
CA_socialDistancingInertiaData = pd.merge(CA_covidcases,maryland_sd_inertia,how='inner',left_on=['FIPS','Date'],right_on=['CTFIPS','date'])
print(CA_socialDistancingInertiaData.shape)

CA_socialDistancingInertiaData.head(2)

(12296, 53)


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases
0,2020-01-22,California,6001,0,0,0,0,0,6001,Alameda County,6,18,20,3.29,29.4,0.3,35.9,0.55,2.74,0,1666753,-2.9,2020-01-22,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0
1,2020-01-23,California,6001,0,0,0,0,0,6001,Alameda County,6,15,19,3.36,29.6,0.3,35.4,0.56,2.80,0,1666753,-0.8,2020-01-23,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0


## Population Density 2020 Data

In [ ]:
pop_density_2020 = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/population-density/Average_Household_Size_and_Population_Density_County.csv', low_memory=False)

In [ ]:
pop_density_2020_california = pop_density_2020[pop_density_2020['State'] == 'California']
pop_density_2020_california.head()

,FID,COUNTYNS,GEOID,ALAND,AWATER,NAME,State,B25010_001E,B25010_001M,B25010_002E,B25010_002M,B25010_003E,B25010_003M,B01001_001E,B01001_001M,B01001_calc_PopDensity,created_user,created_date,last_edited_user,last_edited_date,B01001_calc_PopDensityM,SHAPE_Length,SHAPE_Area
186,187,1675839,6001,1909598013,216923745,Alameda County,California,2.81,0.01,2.93,0.02,2.68,0.02,1643700,NaN,860.757075,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,2.677466,0.199990
187,188,1675840,6003,1912292630,12557304,Alpine County,California,3.71,1.02,3.66,0.97,4.00,2.65,1146,161.0,0.599281,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.084192,2.174214,0.199044
188,189,1675841,6005,1539933577,29470567,Amador County,California,2.42,0.07,2.43,0.08,2.40,0.19,37829,NaN,24.565345,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,2.909839,0.161954
189,190,1675842,6007,4238438186,105311003,Butte County,California,2.55,0.03,2.48,0.04,2.64,0.05,227075,NaN,53.575159,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,4.205815,0.455942
190,191,1675885,6009,2641829200,43797659,Calaveras County,California,2.57,0.07,2.49,0.08,2.83,0.22,45235,NaN,17.122606,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,2.961851,0.276235


In [ ]:
pop_density_2020_california[pop_density_2020_california.eq(6115).any(1)].reset_index()

,index,FID,COUNTYNS,GEOID,ALAND,AWATER,NAME,State,B25010_001E,B25010_001M,B25010_002E,B25010_002M,B25010_003E,B25010_003M,B01001_001E,B01001_001M,B01001_calc_PopDensity,created_user,created_date,last_edited_user,last_edited_date,B01001_calc_PopDensityM,SHAPE_Length,SHAPE_Area
0,243,244,277322,6115,1636913845,31057995,Yuba County,California,2.83,0.04,2.88,0.07,2.75,0.11,75493,NaN,46.119104,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.0,2.832244,0.174096


In [ ]:
CA_socialDistancingInertiaData[CA_socialDistancingInertiaData.eq("Alameda County").any(1)]

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases
0,2020-01-22,California,6001,0,0,0,0,0,6001,Alameda County,6,18,20,3.29,29.4,0.3,35.9,0.55,2.74,0,1666753,-2.9,2020-01-22,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.00,0,1.2,3.9,5.7,0.30,0.00,0.0000,0.00,0,54.000000,0.0,0.00,0.00,0,0
1,2020-01-23,California,6001,0,0,0,0,0,6001,Alameda County,6,15,19,3.36,29.6,0.3,35.4,0.56,2.80,0,1666753,-0.8,2020-01-23,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.00,0,1.2,3.9,5.7,0.30,0.00,0.0000,0.00,0,54.000000,0.0,0.00,0.00,0,0
2,2020-01-24,California,6001,0,0,0,0,0,6001,Alameda County,6,15,19,3.48,29.0,0.3,39.1,0.53,2.94,0,1666753,4.1,2020-01-24,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.00,0,1.2,3.9,5.7,0.30,0.00,0.0000,0.00,0,54.000000,0.0,0.00,0.00,0,0
3,2020-01-25,California,6001,0,0,0,0,0,6001,Alameda County,6,30,24,3.16,26.6,0.3,36.3,0.27,2.88,0,1666753,2.0,2020-01-25,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.00,0,1.2,3.9,5.7,0.30,0.00,0.0000,0.00,1,54.000000,0.0,0.00,0.00,0,0
4,2020-01-26,California,6001,0,0,0,0,0,6001,Alameda County,6,41,28,2.80,25.6,0.4,35.4,0.24,2.56,0,1666753,-9.3,2020-01-26,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.00,0,1.1,3.9,5.7,0.30,0.00,0.0000,0.00,2,54.000000,0.0,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,2020-08-16,California,6001,14973,221,415,2,1,6001,Alameda County,6,51,36,2.67,24.6,0.7,41.0,0.19,2.48,415,1666753,-12.1,2020-08-16,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,48.19,140,5.4,27.3,26.9,1.71,0.58,0.0850,6.27,0,60.731048,6.3,247.70,20.87,1294,4766
208,2020-08-17,California,6001,15123,221,150,0,1,6001,Alameda County,6,47,33,2.60,26.0,0.9,38.5,0.35,2.25,150,1666753,-20.3,2020-08-17,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,48.19,140,5.4,27.3,26.9,1.71,0.58,0.1130,6.22,1,60.762511,6.3,251.13,20.77,1298,4794
209,2020-08-18,California,6001,15437,224,314,3,1,6001,Alameda County,6,43,31,2.74,26.2,0.8,38.4,0.34,2.40,314,1666753,-15.0,2020-08-18,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,48.19,140,5.4,27.3,26.9,1.71,0.58,0.1758,6.28,0,60.788291,6.3,254.04,20.66,1301,5210
210,2020-08-19,California,6001,15836,228,399,4,1,6001,Alameda County,6,45,33,2.69,26.8,0.8,41.9,0.36,2.33,399,1666753,-17.5,2020-08-19,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,48.19,140,5.4,27.3,26.9,1.71,0.58,0.1726,6.23,1,60.948422,6.3,256.36,20.20,1334,5253


In [ ]:
CA_socialDistancingInertiaData_pop = CA_socialDistancingInertiaData

In [ ]:
CA_socialDistancingInertiaData_pop['pop_density_2020'] = 0.00
for index, row in CA_socialDistancingInertiaData_pop.iterrows():
  county_fips = CA_socialDistancingInertiaData_pop['FIPS'][index]
  temp = pop_density_2020_california[pop_density_2020_california.eq(county_fips).any(1)].reset_index()
  CA_socialDistancingInertiaData_pop['pop_density_2020'][index] = temp['B01001_calc_PopDensity'][0]

In [ ]:
print(CA_socialDistancingInertiaData_pop.shape)

CA_socialDistancingInertiaData_pop.head(2)

(12296, 54)


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,mask_rule_active,CTFIPS,CTNAME,STFIPS,Social distancing index,% staying home,Trips/person,% out-of-county trips,% out-of-state trips,Miles/person,Work trips/person,Non-work trips/person,New COVID cases,Population,% change in consumption,date,Transit mode share,% people older than 60,Median income,% African Americans,% Hispanic Americans,% Male,Population density,Employment density,# hot spots/1000 people,Hospital beds/1000 people,ICUs/1000 people,# contact tracing workers/1000 people,# contact tracing workers/1000 people rank,% people older than 60 rank,COVID exposure/1000 people,#days: decreasing ILI cases,Unemployment claims/1000 people,Unemployment rate,% working from home,Cumulative inflation rate,COVID death rate,New cases/1000 people,Active cases/1000 people,#days: decreasing COVID cases,% hospital bed utilization,Testing capacity,Tests done/1000 people,% ICU utilization,Ventilator shortage,Imported COVID cases,pop_density_2020
0,2020-01-22,California,6001,0,0,0,0,0,6001,Alameda County,6,18,20,3.29,29.4,0.3,35.9,0.55,2.74,0,1666753,-2.9,2020-01-22,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,860.757075
1,2020-01-23,California,6001,0,0,0,0,0,6001,Alameda County,6,15,19,3.36,29.6,0.3,35.4,0.56,2.80,0,1666753,-0.8,2020-01-23,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,NaN,12,0.0,0,1.2,3.9,5.7,0.3,0.0,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,860.757075


### Maryland Data file save-CA

In [ ]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/Covid19-data/'
filename = 'CA_socialDistancingInertiaData_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
CA_socialDistancingInertiaData_pop.to_csv(location+filename,index=False)

CA_socialDistancingInertiaData_Feb04.csv


# Feature Selection - Weightage

### Gini index

In [ ]:
CA_All_combined = CA_All_combined.fillna(0)

In [ ]:
def gini(list_of_values):
    sorted_list = sorted(list_of_values)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2.
    return (fair_area - area) / fair_area

In [ ]:
# Calling gini-index method

feature_cols=['fips_x','retail and recreation','grocery and pharmacy',
              'parks','transit stations','workplaces','residential','driving',
              'm50','m50_index','population_density','mask_rule_active','mask_wearing_percent']
gini_values = {}
for col in feature_cols:
  dataset = CA_All_combined[col].values.tolist()
  #print(dataset)
  value = gini(dataset)
  print('cols ',col,' - gini coeff is :',value)
  gini_values[col] = value

In [ ]:
sorted_x = sorted(gini_values.items(), key=lambda kv: kv[1])
sorted_x

## Random Forest - Feature selection

In [ ]:
CA_All_combined.describe().T

In [ ]:
X_cols = ['fips_x','retail and recreation','grocery and pharmacy','parks',
          'transit stations','workplaces','residential','driving','m50','m50_index',
          'population_density','mask_rule_active','mask_wearing_percent']
Y_cols = ['New cases']

In [ ]:
x_train_c = CA_All_combined[X_cols]
y_train_c = CA_All_combined[Y_cols]

In [ ]:
x_train_c

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# create the random forest with your hyperparameters.
rfc = RandomForestClassifier(n_estimators=150)

# fit the model to start training.
rfc.fit(x_train_c, y_train_c)

# get the importance of the resulting features.
f_importances = rfc.feature_importances_

# create a data frame for visualization.
final_df = pd.DataFrame({"Features": x_train_c.columns, "Importances":f_importances})

final_df.sort_values('Importances', ascending=False)

## Decision tree- feature selection

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

clf = DecisionTreeClassifier()
clf = clf.fit(x_train_c, y_train_c)
f=clf.feature_importances_

final_dfs = pd.DataFrame({"Features": x_train_c.columns, "Importances":f})

final_dfs.sort_values('Importances', ascending=False)

## AutoML Implementation

In [ ]:
X_cols = ['fips_x','retail and recreation','grocery and pharmacy','parks',
          'transit stations','workplaces','residential','driving','m50','m50_index',
          'population_density','mask_rule_active','mask_wearing_percent']


In [ ]:
CA_All_combined['Date'] =  pd.to_datetime(CA_All_combined['Date'])

In [ ]:
CA_All_combined.dtypes

In [ ]:
X = CA_All_combined[X_cols]
Y = CA_All_combined[['New cases']]
Y.shape

In [ ]:
X_train,X_test,Y_train, Y_test = train_test_split(X, Y,test_size=0.2, random_state=1)

In [ ]:
num_folds = 12
seed = 7
scoring = 'accuracy'
models = []
models.append(('LR' , LogisticRegression()))
models.append(('LDA' , LinearDiscriminantAnalysis()))
models.append(('KNN' , KNeighborsClassifier()))
models.append(('CART' , DecisionTreeClassifier()))
models.append(('NB' , GaussianNB()))
models.append(('SVM' , SVC()))
models.append(('RF' , RandomForestClassifier(n_estimators=50)))
models.append(('XGBoost', XGBClassifier()))

In [ ]:
Y_train.shape

In [ ]:
results = []
names = []

for name, model in models:
    clf = model
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    accu_score = accuracy_score(Y_test, Y_pred)
    print(name + ": " + str(accu_score))

# Reference 

* https://medium.com/next-gen-machine-learning/feature-selection-best-methods-for-feature-selection-python-f3536aad5b4a